In [1]:
pwd()
using Pkg; Pkg.activate("../../FermiCG/")
using FermiCG, NPZ,JLD2
readdir()

  Activating project at `~/arnab/workspace/FermiCG`


31-element Vector{String}:
 "C_act_loc_sorted_3.molden"
 "README.md"
 "b3lyp-d3_def2svp.xyz"
 "cluster_bases_1.jld2"
 "cluster_bases_2.jld2"
 "clustered1_integrals_h0.npy"
 "clustered1_integrals_h1.npy"
 "clustered1_integrals_h2.npy"
 "clustered1_mo_coeffs.npy"
 "clustered1_overlap_mat.npy"
 ⋮
 "cr_qubit_scf_mo_coeffs.npz.npy"
 "excitations1.xlsx"
 "scf.ipynb"
 "spt_1.jl"
 "spt_2.jl"
 "uhf_density.npy"
 "uhf_mo_coeffs_a.npy"
 "uhf_mo_coeffs_b.npy"
 "wb97x-v_def2svp.xyz"

In [2]:
@load "cmf_data_1.jld2"

4-element Vector{Symbol}:
 :ints
 :clusters
 :init_fspace
 :d1

In [3]:

max_roots = 15

#
# form Cluster data
# cluster_bases = FermiCG.compute_cluster_eigenbasis_spin(ints, clusters, d1, [3,3,3,3,1,4], FockConfig(init_fspace), max_roots=20, verbose=1);
@load "cluster_bases_1.jld2"

clustered_ham = FermiCG.extract_ClusteredTerms(ints, clusters)
cluster_ops = FermiCG.compute_cluster_ops(cluster_bases, ints);
FermiCG.add_cmf_operators!(cluster_ops, cluster_bases, ints, d1.a, d1.b);

check = 0.0
for ci_ops in cluster_ops
    for (opstr, ops) in ci_ops
        for (ftrans, op) in ops
            check += sum(abs.(op))
        end
    end
end



 Remove duplicates
 Number of terms reduced from  2525 to  1105
IDX001:DIM4096:001|002|003|004|005|006|
IDX002:DIM4096:007|008|009|010|011|012|
IDX003:DIM4096:013|014|015|016|017|018|
IDX004:DIM4096:019|020|021|022|023|024|
IDX005:DIM1048576:025|026|027|028|029|030|031|032|033|034|


In [4]:
# Create BST state

v = FermiCG.BSTstate(clusters, FockConfig(init_fspace), cluster_bases)


xspace  = FermiCG.build_compressed_1st_order_state(v, cluster_ops, clustered_ham, nbody=3, thresh=1e-3)
xspace = FermiCG.compress(xspace, thresh=1e-3)
display(size(xspace))

FermiCG.nonorth_add!(v, xspace)
v = FermiCG.BSTstate(v,R=6)
FermiCG.randomize!(v)
FermiCG.orthonormalize!(v)

e_ci, v = FermiCG.ci_solve(v, cluster_ops, clustered_ham)

 In build_compressed_1st_order_state
 Setup threaded jobs:                                0.000250 seconds (1.99 k allocations: 173.094 KiB)
 Number of tasks:                                         661
 Compute matrix-vector:                             12.686385 seconds (47.65 M allocations: 3.212 GiB, 1.82% gc time, 99.18% compilation time)
 Now collect thread results :                        0.010272 seconds (20.63 k allocations: 1.533 MiB, 98.89% compilation time)
 Compressing final σ vector:
   Iter:    1          497 →          497
 |== BST CI ========================================================
 Solve CI with # variables:                               497
 Cache Hamiltonian:                                 11.156340 seconds (24.04 M allocations: 1.610 GiB, 1.33% gc time, 95.01% compilation time)
 Iter:   1 SS: 6    E: -105.38263615  -105.37803888  -105.35851819  -105.34798103  -105.34233995  -105.32580848   R: 1.8e-01  1.7e-01  1.8e-01  1.7e-01  1.7e-01  1.7e-01   LinDep: 1

(497, 1)

([-105.92880640559785, -105.66759517614526, -105.66758036713068, -105.66638342431145, -105.66634981109182, -105.66258409168667], BSTstate{Float64, 5, 6}(MOCluster[MOCluster(1, Int16[1, 2, 3, 4, 5, 6]), MOCluster(2, Int16[7, 8, 9, 10, 11, 12]), MOCluster(3, Int16[13, 14, 15, 16, 17, 18]), MOCluster(4, Int16[19, 20, 21, 22, 23, 24]), MOCluster(5, Int16[25, 26, 27, 28, 29, 30, 31, 32, 33, 34])], OrderedCollections.OrderedDict{FockConfig{5}, OrderedCollections.OrderedDict{TuckerConfig{5}, FermiCG.Tucker{Float64, 5, 6}}}(FockConfig{5}(((3, 3), (3, 3), (3, 3), (3, 3), (5, 5))) => OrderedCollections.OrderedDict(TuckerConfig{5}((1:1, 1:1, 1:1, 1:1, 1:1)) => FermiCG.Tucker{Float64, 5, 6}(([0.9895823562008976;;;;;], [6.527341268016144e-5;;;;;], [-6.429050124755972e-5;;;;;], [9.582185507448372e-6;;;;;], [-6.631256855094013e-5;;;;;], [-0.0014156079371596005;;;;;]), ([1.0;;], [1.0;;], [1.0;;], [1.0;;], [1.0;;])), TuckerConfig{5}((2:15, 1:1, 1:1, 1:1, 2:15)) => FermiCG.Tucker{Float64, 5, 6}(([2.7706

In [5]:
using Printf
display(v)
for ei in 1:length(e_ci)
    @printf(" %5i %12.8f %12.8f eV\n", ei, e_ci[ei]+ints.h0, (e_ci[ei]-e_ci[1])*27.21165)
end


 --------------------------------------------------
 ---------- # Fockspaces -------------------:    53  
 ---------- # Configs    -------------------:   497  
 ---------- Root ---------------------------:     1  
 --------------------------------------------------
 Printing contributions greater than: 0.010000
 Weight              # configs (full)    (α,β)...            
 -------             --------- --------- ----------          
 0.990                                   ( 3,3 )( 3,3 )( 3,3 )( 3,3 )( 5,5 )
     0.990           1         1             1:1    1:1    1:1    1:1    1:1
     0.015           9         196          2:15    1:1    1:1    1:1   2:15
     0.011           4         196          2:15    1:1   2:15    1:1    1:1
     0.019           16        196           1:1   2:15    1:1    1:1   2:15
     0.015           9         196           1:1    1:1    1:1   2:15   2:15
     0.010           1         196           1:1   2:15   2:15    1:1    1:1
     0.010           1 

In [6]:
 e_var, v_var = FermiCG.block_sparse_tucker(v, cluster_ops, clustered_ham,
                                        max_iter    = 20,
                                        nbody       = 4,
                                        H0          = "Hcmf",
                                        thresh_var  = 1e-1,
                                        thresh_foi  = 1e-3,
                                        thresh_pt   = 1e-3,
                                        ci_conv     = 1e-5,
                                        do_pt       = true,                          
                                        resolve_ss  = false,
                                        tol_tucker  = 1e-4)


 max_iter         : 20
 nbody            : 4
 H0               : Hcmf
 thresh_var       : 0.1
 thresh_foi       : 0.001
 thresh_pt        : 0.001
 thresh_spin      : nothing
 ci_conv          : 1.0e-5
 ci_max_iter      : 50
 ci_max_ss_vecs   : 12
 ci_lindep_thresh : 1.0e-10
 resolve_ss       : false
 do_pt            : true
 tol_tucker       : 0.0001



     BST Iteration:    1 epsilon:   0.10000000
 Ref state compressed from:                               497 → 9          (thresh_var  =  1.0e-01)
 Compute zeroth-order energy:                        0.022460 seconds (293.65 k allocations: 18.945 MiB)
 Compute <S^2>:                                      0.005594 seconds (43.64 k allocations: 2.731 MiB, 51.12% compilation time)
  Root       Energy           S2
     1 -105.92340744   2.00001864
     2 -105.66466440   0.31530786
     3 -105.66473679   0.29686074
     4 -105.66361924   2.00208285
     5 -105.66352031   2.00222050
     6 -105.65948622   0.30739415

 In build_compressed_1st_o

┌ Warning: ("initial guess det(v0'v0) = ", 0.9999999999999835)
└ @ FermiCG /Users/arnab/arnab/workspace/FermiCG/src/type_BSTstate.jl:460
┌ Warning: ("initial guess det(v0'v0) = ", 0.9999999999999835)
└ @ FermiCG /Users/arnab/arnab/workspace/FermiCG/src/type_BSTstate.jl:460


  3.556411 seconds (12.37 M allocations: 1.104 GiB, 3.85% gc time, 0.29% compilation time)
 Iter:   1 SS: 6    E: -105.92915902  -105.85260966  -105.80444870  -105.79079643  -105.78948545  -105.77578562   R: 4.2e-03  6.6e-03  2.9e-02  1.3e-02  1.2e-02  1.7e-02   LinDep: 1.8e-15* 
 Iter:   2 SS: 12   E: -105.92918666  -105.85269828  -105.80606136  -105.79119748  -105.78982588  -105.77645556   R: 1.3e-03  2.7e-03  1.4e-02  5.5e-03  5.2e-03  7.9e-03   LinDep: 8.9e-16* 
 Iter:   3 SS: 18   E: -105.92918971  -105.85271496  -105.80658412  -105.79128888  -105.78990168  -105.77660948   R: 4.7e-04  1.3e-03  5.6e-03  3.4e-03  3.1e-03  4.0e-03   LinDep: 1.1e-15* 
 Iter:   4 SS: 24   E: -105.92919014  -105.85271873  -105.80666521  -105.79132130  -105.78992737  -105.77665482   R: 1.7e-04  4.8e-04  2.4e-03  1.8e-03  1.7e-03  2.4e-03   LinDep: 3.8e-15* 
 Iter:   5 SS: 30   E: -105.92919020  -105.85271922  -105.80668075  -105.79133028  -105.78993590  -105.77667173   R: 6.5e-05  2.1e-04  1.3e-03  1.0e-

([-105.92918348394451, -105.852719683894, -105.80675536082082, -105.79854822203166, -105.79131534916964, -105.79011721940715], BSTstate{Float64, 5, 6}(MOCluster[MOCluster(1, Int16[1, 2, 3, 4, 5, 6]), MOCluster(2, Int16[7, 8, 9, 10, 11, 12]), MOCluster(3, Int16[13, 14, 15, 16, 17, 18]), MOCluster(4, Int16[19, 20, 21, 22, 23, 24]), MOCluster(5, Int16[25, 26, 27, 28, 29, 30, 31, 32, 33, 34])], OrderedCollections.OrderedDict{FockConfig{5}, OrderedCollections.OrderedDict{TuckerConfig{5}, FermiCG.Tucker{Float64, 5, 6}}}(FockConfig{5}(((3, 3), (3, 3), (3, 3), (3, 3), (5, 5))) => OrderedCollections.OrderedDict(TuckerConfig{5}((1:1, 1:1, 1:1, 1:1, 1:1)) => FermiCG.Tucker{Float64, 5, 6}(([-0.9880336778402812;;;;;], [8.948697249490185e-6;;;;;], [0.01633754256679842;;;;;], [-9.74363454611752e-5;;;;;], [-0.0009484809768770308;;;;;], [-0.0022020949641427916;;;;;]), ([-1.0;;], [-1.0;;], [-1.0;;], [-1.0;;], [-1.0;;])), TuckerConfig{5}((1:1, 1:1, 1:1, 1:1, 2:15)) => FermiCG.Tucker{Float64, 5, 6}(([5.56

In [7]:
v1 = deepcopy(v_var);
display(v1,root=1)


 --------------------------------------------------
 ---------- # Fockspaces -------------------:   212  
 ---------- # Configs    -------------------:  3073  
 ---------- Root ---------------------------:     1  
 --------------------------------------------------
 Printing contributions greater than: 0.010000
 Weight              # configs (full)    (α,β)...            
 -------             --------- --------- ----------          
 0.989                                   ( 3,3 )( 3,3 )( 3,3 )( 3,3 )( 5,5 )
     0.988           1         1             1:1    1:1    1:1    1:1    1:1
     0.016           70        196          2:15    1:1    1:1    1:1   2:15
     0.011           4         196          2:15    1:1   2:15    1:1    1:1
     0.020           104       196           1:1    1:1   2:15    1:1   2:15
     0.020           88        196           1:1   2:15    1:1    1:1   2:15
     0.016           84        196           1:1    1:1    1:1   2:15   2:15
     0.010           1 

In [9]:
 e2, v2 = FermiCG.block_sparse_tucker(v1, cluster_ops, clustered_ham,
                                        max_iter    = 20,
                                        nbody       = 4,
                                        H0          = "Hcmf",
                                        thresh_var  = 5e-2,
                                        thresh_foi  = 1e-4,
                                        thresh_pt   = 1e-3,
                                        ci_conv     = 1e-5,
                                        do_pt       = true,                          
                                        resolve_ss  = false,
                                        tol_tucker  = 1e-4)

 max_iter         : 20
 nbody            : 4
 H0               : Hcmf
 thresh_var       : 0.05
 thresh_foi       : 0.0001
 thresh_pt        : 0.001
 thresh_spin      : nothing
 ci_conv          : 1.0e-5
 ci_max_iter      : 50
 ci_max_ss_vecs   : 12
 ci_lindep_thresh : 1.0e-10
 resolve_ss       : false
 do_pt            : true
 tol_tucker       : 0.0001



     BST Iteration:    1 epsilon:   0.05000000
 Ref state compressed from:                              3073 → 61         (thresh_var  =  5.0e-02)
 Compute zeroth-order energy:                        0.176966 seconds (1.19 M allocations: 88.486 MiB, 16.33% gc time)
 Compute <S^2>:                                      0.009165 seconds (159.34 k allocations: 10.275 MiB)
  Root       Energy           S2
     1 -105.92452746   1.99994343
     2 -105.84980431   0.01703419
     3 -105.80270168   2.00013917
     4 -105.79365218   1.16314911
     5 -105.78573080   1.99079593
     6 -105.78460202   1.99743544

 In build_compressed_1st_order_st

┌ Warning: ("initial guess det(v0'v0) = ", 0.9999999999999868)
└ @ FermiCG /Users/arnab/arnab/workspace/FermiCG/src/type_BSTstate.jl:460


  5.571890 seconds (13.83 M allocations: 1.426 GiB, 7.40% gc time)
 Iter:   1 SS: 6    E: -105.92934817  -105.85303525  -105.81113553  -105.80742726  -105.79612231  -105.79185469   R: 2.7e-03  3.3e-03  1.8e-02  4.2e-03  1.4e-02  1.0e-02   LinDep: 4.4e-16* 
 Iter:   2 SS: 12   E: -105.92935918  -105.85305311  -105.81271783  -105.80746423  -105.79843574  -105.79263247   R: 6.1e-04  7.8e-04  2.3e-02  2.2e-03  3.0e-02  1.3e-02   LinDep: 8.9e-16* 
 Iter:   3 SS: 18   E: -105.92935977  -105.85305423  -105.81510665  -105.80748241  -105.80223093  -105.79315929   R: 2.4e-04  3.4e-04  2.0e-02  1.7e-03  2.3e-02  7.4e-03   LinDep: 7.8e-15* 
 Iter:   4 SS: 24   E: -105.92935988  -105.85305451  -105.81617419  -105.80749077  -105.80355224  -105.79330222   R: 9.8e-05  1.8e-04  1.4e-02  1.3e-03  1.5e-02  4.9e-03   LinDep: 2.3e-15* 
 Iter:   5 SS: 30   E: -105.92935990  -105.85305458  -105.81672874  -105.80749596  -105.80417458  -105.79337417   R: 4.0e-05  9.5e-05  7.8e-03  8.0e-04  8.2e-03  2.9e-03   L

┌ Warning: ("initial guess det(v0'v0) = ", 0.999999999999978)
└ @ FermiCG /Users/arnab/arnab/workspace/FermiCG/src/type_BSTstate.jl:460
┌ Warning: ("initial guess det(v0'v0) = ", 0.999999999999978)
└ @ FermiCG /Users/arnab/arnab/workspace/FermiCG/src/type_BSTstate.jl:460


  5.293160 seconds (14.98 M allocations: 1.543 GiB, 7.82% gc time)
 Iter:   1 SS: 6    E: -105.92935681  -105.85306253  -105.81234128  -105.80747254  -105.80450498  -105.78954795   R: 2.3e-03  2.1e-03  1.4e-02  3.2e-03  6.3e-03  1.5e-02   LinDep: 3.1e-15* 
 Iter:   2 SS: 12   E: -105.92936364  -105.85307297  -105.81578566  -105.80750159  -105.80499160  -105.79272374   R: 5.3e-04  1.1e-03  3.3e-02  2.2e-03  1.1e-02  3.1e-02   LinDep: 1.9e-15* 
 Iter:   3 SS: 18   E: -105.92936413  -105.85307563  -105.81917250  -105.80751491  -105.80536796  -105.79589079   R: 1.9e-04  5.6e-04  2.3e-02  1.4e-03  7.7e-03  2.2e-02   LinDep: 6.0e-15* 
 Iter:   4 SS: 24   E: -105.92936420  -105.85307649  -105.82084672  -105.80752179  -105.80557139  -105.79745842   R: 6.6e-05  3.4e-04  1.6e-02  1.1e-03  5.8e-03  1.5e-02   LinDep: 1.2e-14* 
 Iter:   5 SS: 30   E: -105.92936420  -105.85307675  -105.82151462  -105.80752505  -105.80566784  -105.79813129   R: 2.1e-05  1.8e-04  9.2e-03  6.7e-04  3.7e-03  9.5e-03   L

...

┌ Warning: ("initial guess det(v0'v0) = ", 1.0000000000000158)
└ @ FermiCG /Users/arnab/arnab/workspace/FermiCG/src/type_BSTstate.jl:460


  5.595945 seconds (13.60 M allocations: 1.392 GiB, 8.60% gc time, 0.22% compilation time)
 Iter:   1 SS: 6    E: -105.92937043  -105.85308989  -105.83089618  -105.80753152  -105.80572131  -105.78200989   R: 1.1e-03  1.3e-03  4.2e-03  2.1e-03  2.3e-03  1.8e-02   LinDep: 4.4e-16* 
 Iter:   2 SS: 12   E: -105.92937237  -105.85309284  -105.83094715  -105.80753948  -105.80573662  -105.78318118   R: 2.4e-04  4.3e-04  1.9e-03  5.4e-04  9.9e-04  9.9e-03   LinDep: 6.4e-15* 
 Iter:   3 SS: 18   E: -105.92937246  -105.85309326  -105.83095751  -105.80754011  -105.80573941  -105.78356945   R: 8.4e-05  2.2e-04  1.1e-03  2.3e-04  6.0e-04  8.6e-03   LinDep: 1.1e-14* 
 Iter:   4 SS: 24   E: -105.92937248  -105.85309339  -105.83096057  -105.80754023  -105.80574055  -105.78392163   R: 3.3e-05  1.4e-04  5.6e-04  1.2e-04  4.1e-04  9.2e-03   LinDep: 8.9e-15* 
 Iter:   5 SS: 30   E: -105.92937248  -105.85309343  -105.83096144  -105.80754028  -105.80574114  -105.78438899   R: 1.5e-05  7.0e-05  3.5e-04  8.7e-

┌ Warning: ("initial guess det(v0'v0) = ", 1.0000000000000149)
└ @ FermiCG /Users/arnab/arnab/workspace/FermiCG/src/type_BSTstate.jl:460
┌ Warning: ("initial guess det(v0'v0) = ", 1.000000000000015)
└ @ FermiCG /Users/arnab/arnab/workspace/FermiCG/src/type_BSTstate.jl:460


  4.913086 seconds (13.57 M allocations: 1.381 GiB, 9.14% gc time)
 Iter:   1 SS: 6    E: -105.92935068  -105.85309833  -105.83603188  -105.80748716  -105.80602201  -105.79539520   R: 2.2e-03  2.7e-03  9.7e-03  3.4e-03  4.2e-03  1.6e-02   LinDep: 3.0e-15* 
 Iter:   2 SS: 12   E: -105.92935864  -105.85311083  -105.83625952  -105.80750753  -105.80605119  -105.79584060   R: 4.9e-04  7.5e-04  5.0e-03  9.7e-04  1.2e-03  4.8e-03   LinDep: 1.1e-15* 
 Iter:   3 SS: 18   E: -105.92935905  -105.85311202  -105.83631201  -105.80750948  -105.80605485  -105.79590515   R: 2.0e-04  4.0e-04  2.3e-03  4.5e-04  6.0e-04  2.5e-03   LinDep: 1.3e-15* 
 Iter:   4 SS: 24   E: -105.92935913  -105.85311240  -105.83632495  -105.80750999  -105.80605579  -105.79592175   R: 8.5e-05  2.3e-04  1.2e-03  2.6e-04  3.3e-04  1.2e-03   LinDep: 8.9e-15* 
 Iter:   5 SS: 30   E: -105.92935914  -105.85311253  -105.83632825  -105.80751018  -105.80605609  -105.79592516   R: 3.5e-05  1.3e-04  7.2e-04  1.8e-04  2.2e-04  8.1e-04   L

([-105.92936062367441, -105.85311322629359, -105.83636226586269, -105.80751641266819, -105.80606084490178, -105.79597939623453], BSTstate{Float64, 5, 6}(MOCluster[MOCluster(1, Int16[1, 2, 3, 4, 5, 6]), MOCluster(2, Int16[7, 8, 9, 10, 11, 12]), MOCluster(3, Int16[13, 14, 15, 16, 17, 18]), MOCluster(4, Int16[19, 20, 21, 22, 23, 24]), MOCluster(5, Int16[25, 26, 27, 28, 29, 30, 31, 32, 33, 34])], OrderedCollections.OrderedDict{FockConfig{5}, OrderedCollections.OrderedDict{TuckerConfig{5}, FermiCG.Tucker{Float64, 5, 6}}}(FockConfig{5}(((3, 3), (3, 3), (3, 3), (3, 3), (5, 5))) => OrderedCollections.OrderedDict(TuckerConfig{5}((1:1, 1:1, 1:1, 1:1, 1:1)) => FermiCG.Tucker{Float64, 5, 6}(([-0.9875397964965529;;;;;], [6.21922302170746e-8;;;;;], [2.285801833607181e-5;;;;;], [-0.017350778955270065;;;;;], [-0.0017388278132306945;;;;;], [0.0005038091054601511;;;;;]), ([-1.0;;], [-1.0;;], [-1.0;;], [-1.0;;], [-1.0;;])), TuckerConfig{5}((1:1, 1:1, 1:1, 1:1, 2:15)) => FermiCG.Tucker{Float64, 5, 6}(([1.

In [10]:
display(v2, root=3, thresh=.1)



 --------------------------------------------------
 ---------- # Fockspaces -------------------:   251  
 ---------- # Configs    -------------------:  6134  
 ---------- Root ---------------------------:     3  
 --------------------------------------------------
 Printing contributions greater than: 0.100000
 Weight              # configs (full)    (α,β)...            
 -------             --------- --------- ----------          
 0.966                                   ( 3,3 )( 3,3 )( 3,3 )( 3,3 )( 5,5 )
     0.964           5         14            1:1    1:1    1:1    1:1   2:15
 ---------                               ----------          
 --------------------------------------------------



In [11]:
 e3, v3 = FermiCG.block_sparse_tucker(v2, cluster_ops, clustered_ham,
                                        max_iter    = 20,
                                        nbody       = 4,
                                        H0          = "Hcmf",
                                        thresh_var  = 2e-2,
                                        thresh_foi  = 1e-5,
                                        thresh_pt   = 1e-4,
                                        ci_conv     = 1e-5,
                                        do_pt       = true,                          
                                        resolve_ss  = false,
                                        tol_tucker  = 1e-4)

 max_iter         : 20
 nbody            : 4
 H0               : Hcmf
 thresh_var       : 0.02
 thresh_foi       : 1.0e-5
 thresh_pt        : 0.0001
 thresh_spin      : nothing
 ci_conv          : 1.0e-5
 ci_max_iter      : 50
 ci_max_ss_vecs   : 12
 ci_lindep_thresh : 1.0e-10
 resolve_ss       : false
 do_pt            : true
 tol_tucker       : 0.0001



     BST Iteration:    1 epsilon:   0.02000000
 Ref state compressed from:                              6134 → 132        (thresh_var  =  2.0e-02)
 Compute zeroth-order energy:                        0.390327 seconds (2.69 M allocations: 198.337 MiB, 13.20% gc time)
 Compute <S^2>:                                      0.042071 seconds (325.20 k allocations: 21.275 MiB, 53.28% gc time)
  Root       Energy           S2
     1 -105.92561609   2.00105281
     2 -105.85043018   0.00950673
     3 -105.83365474   0.27022890
     4 -105.80446684   1.99968658
     5 -105.80222168   1.97884087
     6 -105.79208162   2.10255772

 In build_compr

┌ Warning: ("initial guess det(v0'v0) = ", 1.0000000000000526)
└ @ FermiCG /Users/arnab/arnab/workspace/FermiCG/src/type_BSTstate.jl:460
┌ Warning: ("initial guess det(v0'v0) = ", 1.00000000000153)
└ @ FermiCG /Users/arnab/arnab/workspace/FermiCG/src/type_BSTstate.jl:460
┌ Warning: ("initial guess det(v0'v0) = ", 1.0000000000015303)
└ @ FermiCG /Users/arnab/arnab/workspace/FermiCG/src/type_BSTstate.jl:460


 Cache Hamiltonian:                                 74.335455 seconds (104.70 M allocations: 33.269 GiB, 14.26% gc time, 0.01% compilation time)
 Iter:   1 SS: 6    E: -105.92936210  -105.85311655  -105.83637073  -105.80752045  -105.80606586  -105.79598810   R: 1.2e-02  1.1e-02  1.7e-02  1.4e-02  1.5e-02  1.7e-02   LinDep: 7.8e-15* 


In [ ]:
 e4, v4 = FermiCG.block_sparse_tucker(v3, cluster_ops, clustered_ham,
                                        max_iter    = 20,
                                        max_iter_pt = 200,
                                        nbody       = 4,
                                        H0          = "Hcmf",
                                        thresh_var  = 1e-2,
                                        thresh_foi  = 1e-5,
                                        thresh_pt   = 1e-4,
                                        ci_conv     = 1e-5,
                                        do_pt       = true,                          
                                        resolve_ss  = false,
                                        tol_tucker  = 1e-4)